In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

In [2]:
# load the dataset 
df = pd.read_csv('/Users/tsconing/Downloads/FIC.Full.csv')

# print(df.describe())
df.head()

,Age,Age.Group,Gender,Locality,Marital status,Life.Style,Sleep,Category,Depression,Hyperlipi,...,oldpeak,slope,ca,thal,num,SK,SK.React,Reaction,Mortality,Follow.Up
0,45,41-50,Female,RURAL,MARRIED,NO,NO,FREE,YES,YES,...,3.0,2,0,7,2,1,NO,0,0,60
1,51,51-60,Female,URBAN,MARRIED,NO,NO,FREE,YES,YES,...,1.2,2,0,7,2,1,NO,0,0,15
2,55,51-60,Female,RURAL,MARRIED,YES,YES,FREE,YES,YES,...,3.4,2,0,3,2,1,NO,0,0,6
3,55,51-60,Female,RURAL,MARRIED,YES,YES,FREE,YES,YES,...,2.0,2,1,7,3,1,NO,0,0,52
4,56,51-60,Female,RURAL,MARRIED,YES,NO,FREE,YES,YES,...,4.0,3,2,7,3,1,NO,0,0,34


In [3]:
print(df[df.isna().any(axis=1)])

Empty DataFrame
Columns: [Age, Age.Group, Gender, Locality  , Marital status                       , Life.Style                                                                              , Sleep, Category, Depression, Hyperlipi, Smoking, Family.History, F.History, Diabetes, HTN, Allergies, BP, Thrombolysis, BGR, B.Urea, S.Cr, S.Sodium, S.Potassium, S.Chloride, C.P.K, CK.MB, ESR, WBC, RBC, Hemoglobin, P.C.V, M.C.V, M.C.H, M.C.H.C, PLATELET_COUNT, NEUTROPHIL, LYMPHO, MONOCYTE, EOSINO, Others , CO, Diagnosis, Hypersensitivity, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal, num, SK, SK.React, Reaction, Mortality, Follow.Up]
Index: []

[0 rows x 60 columns]


In [4]:
df.columns = df.columns.str.strip()


print(df.shape)

df = df.drop(columns=['Others','Family.History', 'CO', 'Diagnosis', 'Life.Style', 'Sleep', 'Category', 'Age.Group'])
print(df.shape)

(368, 60)
(368, 52)


In [5]:
# turn the binary data into 1/0s 
one_hot_encoded = pd.get_dummies(df[['Gender', 'Locality', 'Marital status', 'Depression', 'Hyperlipi', 'Smoking', 'HTN', 'Allergies', 'Hypersensitivity', 'SK.React']])
# drop categories 
df = df.drop(columns=['Gender', 'Locality', 'Marital status', 'Depression', 'Hyperlipi', 'Smoking', 'HTN', 'Allergies', 'Hypersensitivity', 'SK.React'])
# add categories back to dataframe 
df = pd.concat([df, one_hot_encoded], axis=1)
print(df.shape)


(368, 67)


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# initialize the MinMax scaler 
scaler = MinMaxScaler()

attr = df.drop('Mortality', axis=1)

target = df['Mortality']

attr = pd.DataFrame(scaler.fit_transform(attr), columns=attr.columns)
attr.head()


,Age,F.History,Diabetes,BP,Thrombolysis,BGR,B.Urea,S.Cr,S.Sodium,S.Potassium,...,Allergies_YES,Hypersensitivity_NO,Hypersensitivity_YES,SK.React_BODY.PAIN,SK.React_COUGH.BLEEDING,SK.React_LUNGS,SK.React_NAUSEA.TEMP,SK.React_NO,SK.React_SKIN.BLEEDING,SK.React_STOMACH.BLEEDING
0,0.396226,0.0,1.0,0.183377,0.0,0.047714,0.065611,0.013453,0.529412,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.509434,0.0,0.0,0.092145,0.0,0.149105,0.037529,0.004484,0.882353,0.70,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.584906,0.0,1.0,0.184290,0.0,0.170974,0.088588,0.017937,0.470588,0.45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.584906,0.0,1.0,0.726211,0.0,0.170974,0.088588,0.017937,0.470588,0.45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.603774,0.0,1.0,0.092145,0.0,0.049702,0.193260,0.026906,0.588235,0.60,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
kmeans = KMeans(n_clusters=2, random_state=10)

kmeans.fit(attr)

labels = kmeans.labels_
centers = kmeans.cluster_centers_



/opt/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
df_cluster = pd.DataFrame({'Actual':target, 'Cluster':labels})

print(df_cluster.head())

cross_tabulation = pd.crosstab(df_cluster['Actual'], df_cluster['Cluster'])

print(cross_tabulation)

   Actual  Cluster
0       0        1
1       0        1
2       0        1
3       0        1
4       0        1
Cluster    0   1
Actual          
0        238  50
1         43  37


In [9]:
count = (df_cluster['Actual'] == df_cluster['Cluster']).sum()


print(f'This is how many rows are accurate after being scaled by MinMaxScaler in euclidean clustering : {count} \nOut of {len(df_cluster)} \nWhich is an accuracy rate of {(count/len(df_cluster)) * 100}%')
print()

This is how many rows are accurate after being scaled by MinMaxScaler in euclidean clustering : 275 
Out of 368 
Which is an accuracy rate of 74.72826086956522%

